In [43]:
import pandas as pd
import glob
import os
from sqlalchemy import create_engine

os.listdir("bases/")

['residencias.csv',
 'media_preco.csv',
 'airbnb_ny_2019.csv',
 'mapeamento_vizinhanca.csv']

In [105]:
airbnb = pd.read_csv("bases/airbnb_ny_2019.csv")

In [5]:
mapviz = pd.read_csv('bases/mapeamento_vizinhanca.csv', sep=';')

In [6]:
f_airbnb = airbnb.drop_duplicates()

In [7]:
f_airbnb = f_airbnb.dropna()

In [8]:
f_mapviz = mapviz.drop_duplicates()

In [9]:
f_mapviz = f_mapviz.dropna()

In [13]:
new_airbnb = pd.merge(left=f_airbnb, right=f_mapviz, how='left', left_on='neighbourhood', right_on='vizinhanca')

In [15]:
del new_airbnb['vizinhanca']

In [29]:
new_airbnb.rename(columns={"vizinhanca_grupo":"neighbourhood_group"}, inplace=True)

In [72]:
res = new_airbnb[new_airbnb.neighbourhood_group.isin(['Brooklyn', 'Manhattan', 'Queens', 'Staten Island'])]

In [73]:
res.to_csv('bases/residencias.csv', index=False)

In [106]:
res = pd.read_csv('bases/residencias.csv', usecols=['neighbourhood_group', 'room_type','price'])

In [107]:
means = res.groupby(['neighbourhood_group', 'room_type']).mean()

In [83]:
means.to_csv('bases/media_preco.csv')

---
*A partir daqui, ocorre a carga de um Banco de dados SQLite para uso da API*

---

In [97]:
res = pd.read_csv('bases/residencias.csv')

In [98]:
engine = create_engine('sqlite:///titau/tItau.db', echo=True)

In [99]:
dbconn = engine.connect()

2020-08-03 14:15:08,504 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-03 14:15:08,514 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 14:15:08,524 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-03 14:15:08,530 INFO sqlalchemy.engine.base.Engine ()


In [100]:
res.to_sql('residencias', dbconn, if_exists='fail', index=False)

2020-08-03 14:15:13,821 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("residencias")
2020-08-03 14:15:13,822 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 14:15:13,823 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("residencias")
2020-08-03 14:15:13,824 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 14:15:13,827 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE residencias (
	id BIGINT, 
	name TEXT, 
	host_id BIGINT, 
	host_name TEXT, 
	neighbourhood TEXT, 
	latitude FLOAT, 
	longitude FLOAT, 
	room_type TEXT, 
	price BIGINT, 
	minimum_nights BIGINT, 
	number_of_reviews BIGINT, 
	last_review TEXT, 
	reviews_per_month FLOAT, 
	calculated_host_listings_count BIGINT, 
	availability_365 BIGINT, 
	neighbourhood_group TEXT
)


2020-08-03 14:15:13,829 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 14:15:14,052 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-03 14:15:14,123 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-03 14:15:14,840 INFO sqlalchem

In [108]:
means = pd.read_csv('bases/media_preco.csv')

In [109]:
means.to_sql('medias', dbconn, if_exists='fail',  index_label='id')

2020-08-03 16:53:15,810 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("medias")
2020-08-03 16:53:15,813 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 16:53:15,844 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("medias")
2020-08-03 16:53:15,846 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 16:53:15,849 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE medias (
	id BIGINT, 
	neighbourhood_group TEXT, 
	room_type TEXT, 
	price FLOAT
)


2020-08-03 16:53:15,851 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 16:53:16,159 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-03 16:53:16,160 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_medias_id ON medias (id)
2020-08-03 16:53:16,161 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 16:53:16,337 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-03 16:53:16,365 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-03 16:53:16,366 INFO sqlalchemy.engine.base.Engine INSERT INTO medias (id, neighbourhood_group

In [103]:
sql_likes = '''
CREATE TABLE likes (
	id BIGINT, 
	residencia_id BIGINT,
    like INT default 1
)
'''
dbconn.execute(sql_likes)

2020-08-03 14:15:50,275 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE likes (
	id BIGINT, 
	residencia_id BIGINT,
    like INT default 1
)

2020-08-03 14:15:50,284 INFO sqlalchemy.engine.base.Engine ()
2020-08-03 14:15:50,574 INFO sqlalchemy.engine.base.Engine COMMIT
